# AUTOENCODER

In [5]:
import os
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
%matplotlib inline

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 200
batch_size = 128
learning_rate = 1e-3


def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
#         print(img)
#         print(img.size())
#         print('--------------------------')
        img = img.view(img.size(0), -1)
#         print(img)
#         print(img.size())
#         print('--------------------------')
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        #MIDIMG=model.encoder(img).view(img.size(0),8,8).cpu().detach().numpy()
        #MIDIMG=MIDIMG[0]
        #MIDIMG=np.rint(MIDIMG/np.sum(MIDIMG)*255)
        #print(MIDIMG)
        #plt.imshow(MIDIMG)
        #plt.show()
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    if epoch % 20 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        save_image(x, './mlp_img/x_{}.png'.format(epoch))
        save_image(x_hat, './mlp_img/x_hat_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

E:\Installations\Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:105: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/200], loss:0.1157, MSE_loss:0.0363
epoch [2/200], loss:0.0862, MSE_loss:0.0261
epoch [3/200], loss:0.0709, MSE_loss:0.0214
epoch [4/200], loss:0.0645, MSE_loss:0.0192
epoch [5/200], loss:0.0542, MSE_loss:0.0159
epoch [6/200], loss:0.0538, MSE_loss:0.0159
epoch [7/200], loss:0.0546, MSE_loss:0.0161
epoch [8/200], loss:0.0542, MSE_loss:0.0160
epoch [9/200], loss:0.0512, MSE_loss:0.0151
epoch [10/200], loss:0.0499, MSE_loss:0.0145
epoch [11/200], loss:0.0467, MSE_loss:0.0138
epoch [12/200], loss:0.0431, MSE_loss:0.0125
epoch [13/200], loss:0.0425, MSE_loss:0.0126
epoch [14/200], loss:0.0424, MSE_loss:0.0124
epoch [15/200], loss:0.0386, MSE_loss:0.0112
epoch [16/200], loss:0.0431, MSE_loss:0.0126
epoch [17/200], loss:0.0426, MSE_loss:0.0123
epoch [18/200], loss:0.0381, MSE_loss:0.0112
epoch [19/200], loss:0.0373, MSE_loss:0.0108
epoch [20/200], loss:0.0402, MSE_loss:0.0118
epoch [21/200], loss:0.0368, MSE_loss:0.0106
epoch [22/200], loss:0.0386, MSE_loss:0.0113
epoch [23/200], los

epoch [182/200], loss:0.0264, MSE_loss:0.0075
epoch [183/200], loss:0.0252, MSE_loss:0.0071
epoch [184/200], loss:0.0226, MSE_loss:0.0063
epoch [185/200], loss:0.0237, MSE_loss:0.0068
epoch [186/200], loss:0.0229, MSE_loss:0.0064
epoch [187/200], loss:0.0223, MSE_loss:0.0065
epoch [188/200], loss:0.0232, MSE_loss:0.0066
epoch [189/200], loss:0.0255, MSE_loss:0.0072
epoch [190/200], loss:0.0234, MSE_loss:0.0065
epoch [191/200], loss:0.0231, MSE_loss:0.0066
epoch [192/200], loss:0.0255, MSE_loss:0.0073
epoch [193/200], loss:0.0266, MSE_loss:0.0076
epoch [194/200], loss:0.0240, MSE_loss:0.0067
epoch [195/200], loss:0.0233, MSE_loss:0.0066
epoch [196/200], loss:0.0221, MSE_loss:0.0062
epoch [197/200], loss:0.0229, MSE_loss:0.0064
epoch [198/200], loss:0.0243, MSE_loss:0.0069
epoch [199/200], loss:0.0218, MSE_loss:0.0061
epoch [200/200], loss:0.0221, MSE_loss:0.0062


# Encoded IMAGE

In [ ]:
import os
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
%matplotlib inline

if not os.path.exists('./mlp_pansharpened'):
    os.mkdir('./mlp_pansharpened')


def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 500
batch_size = 128
learning_rate = 1e-3


def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 1764),
            nn.ReLU(True),
            nn.Linear(1764, 3136),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(3136, 1764),
            nn.ReLU(True),
            nn.Linear(1764, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
#         print(img)
#         print(img.size())
#         print('--------------------------')
        img = img.view(img.size(0), -1)
#         print(img)
#         print(img.size())
#         print('--------------------------')
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
#         MIDIMG=model.encoder(img).view(img.size(0),56,56).cpu().detach().numpy()
#         MIDIMG=MIDIMG[0]
#         MIDIMG=np.rint(MIDIMG/np.sum(MIDIMG)*255)
#         print(MIDIMG)
#         plt.imshow(MIDIMG)
#         plt.show()
        
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    if epoch % 1 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        MIDIMG= model.encoder(img).view(img.size(0),1,56,56).cpu().data
        save_image(x, './mlp_pansharpened/x_{}.png'.format(epoch))
        save_image(x_hat, './mlp_pansharpened/x_hat_{}.png'.format(epoch))
        save_image(MIDIMG, './mlp_pansharpened/x_midlayer{}.png'.format(epoch))
        save_image(img.view(img.size(0),1,28,28).cpu().data, './mlp_pansharpened/x_inputlayer{}.png'.format(epoch))
torch.save(model.state_dict(), './sim_autoencoder.pth')

E:\Installations\Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:106: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/500], loss:0.0573, MSE_loss:0.0170
epoch [2/500], loss:0.0446, MSE_loss:0.0130
epoch [3/500], loss:0.0340, MSE_loss:0.0099
epoch [4/500], loss:0.0335, MSE_loss:0.0096
epoch [5/500], loss:0.0259, MSE_loss:0.0074
epoch [6/500], loss:0.0232, MSE_loss:0.0067
epoch [7/500], loss:0.0212, MSE_loss:0.0060
epoch [8/500], loss:0.0205, MSE_loss:0.0058
epoch [9/500], loss:0.0230, MSE_loss:0.0066
epoch [10/500], loss:0.0199, MSE_loss:0.0056
epoch [11/500], loss:0.0189, MSE_loss:0.0053
epoch [12/500], loss:0.0204, MSE_loss:0.0057
epoch [13/500], loss:0.0192, MSE_loss:0.0055
epoch [14/500], loss:0.0172, MSE_loss:0.0048
epoch [15/500], loss:0.0188, MSE_loss:0.0052
epoch [16/500], loss:0.0163, MSE_loss:0.0044
epoch [17/500], loss:0.0154, MSE_loss:0.0042
epoch [18/500], loss:0.0186, MSE_loss:0.0053
epoch [19/500], loss:0.0146, MSE_loss:0.0040
epoch [20/500], loss:0.0172, MSE_loss:0.0048
epoch [21/500], loss:0.0158, MSE_loss:0.0044
epoch [22/500], loss:0.0156, MSE_loss:0.0044
epoch [23/500], los

# Denoise

In [40]:
import os
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
%matplotlib inline

if not os.path.exists('./AE_denoise'):
    os.mkdir('./AE_denoise')


def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 50
batch_size = 128
learning_rate = 1e-3


def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))


def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
#test_dataset=MNIST('./data', transform=img_transform, train = False, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 196),
            nn.ReLU(True),
            nn.Linear(196, 16),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(16, 196),
            nn.ReLU(True),
            nn.Linear(196, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
#         print(img)
#         print(img.size())
#         print('--------------------------')
        img = img.view(img.size(0), -1)
        noise = (torch.randint(-10,2,(img.size(0),img.size(1)))>0).float()
        noised_img=((img+noise)>0).float()
#         print(img)
#         print(img.size())
#         print('--------------------------')
        noised_img = Variable(noised_img).cuda()
        
        # ===================forward=====================
        output = model(noised_img)
#         MIDIMG=model.encoder(img).view(img.size(0),56,56).cpu().detach().numpy()
#         MIDIMG=MIDIMG[0]
#         MIDIMG=np.rint(MIDIMG/np.sum(MIDIMG)*255)
#         print(MIDIMG)
#         plt.imshow(MIDIMG)
#         plt.show()
        
        loss = criterion(output, noised_img)
        MSE_loss = nn.MSELoss()(output, noised_img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    if (epoch+1) % 10 == 0 or epoch == 0:
        x_hat = to_img(output.cpu().data)
        #MIDIMG= model.encoder(img).view(img.size(0),1,56,56).cpu().data
        save_image(x_hat, './AE_denoise/c_output_{}.png'.format(epoch+1))
        #save_image(MIDIMG, './mlp_pansharpened/x_midlayer{}.png'.format(epoch))
        save_image(img.view(img.size(0),1,28,28).cpu().data, './AE_denoise/a_raw{}.png'.format(epoch+1))
        save_image(noised_img.view(img.size(0),1,28,28).cpu().data, './AE_denoise/b_noised{}.png'.format(epoch+1))
        
        
        
for epoch_2 in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()

        output = model(img)

        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch_2 [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch_2 + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    if (epoch_2+1) % 10 == 0 or epoch_2 == 0:
        x_hat = to_img(output.cpu().data)
        save_image(x_hat, './AE_denoise/d_output_{}.png'.format(epoch_2+1))







E:\Installations\Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:110: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/50], loss:0.3814, MSE_loss:0.1137
epoch [2/50], loss:0.3699, MSE_loss:0.1094
epoch [3/50], loss:0.3611, MSE_loss:0.1059
epoch [4/50], loss:0.3529, MSE_loss:0.1029
epoch [5/50], loss:0.3532, MSE_loss:0.1030
epoch [6/50], loss:0.3514, MSE_loss:0.1022
epoch [7/50], loss:0.3465, MSE_loss:0.1006
epoch [8/50], loss:0.3448, MSE_loss:0.0998
epoch [9/50], loss:0.3469, MSE_loss:0.1005
epoch [10/50], loss:0.3469, MSE_loss:0.1004
epoch [11/50], loss:0.3477, MSE_loss:0.1007
epoch [12/50], loss:0.3457, MSE_loss:0.1000
epoch [13/50], loss:0.3411, MSE_loss:0.0981
epoch [14/50], loss:0.3415, MSE_loss:0.0986
epoch [15/50], loss:0.3452, MSE_loss:0.0996
epoch [16/50], loss:0.3418, MSE_loss:0.0985
epoch [17/50], loss:0.3401, MSE_loss:0.0980
epoch [18/50], loss:0.3426, MSE_loss:0.0987
epoch [19/50], loss:0.3444, MSE_loss:0.0993
epoch [20/50], loss:0.3408, MSE_loss:0.0981
epoch [21/50], loss:0.3413, MSE_loss:0.0984
epoch [22/50], loss:0.3428, MSE_loss:0.0988
epoch [23/50], loss:0.3375, MSE_loss:0.09

E:\Installations\Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:137: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch_2 [1/50], loss:0.1047, MSE_loss:0.0323
epoch_2 [2/50], loss:0.1022, MSE_loss:0.0316
epoch_2 [3/50], loss:0.1008, MSE_loss:0.0313
epoch_2 [4/50], loss:0.0999, MSE_loss:0.0310
epoch_2 [5/50], loss:0.0993, MSE_loss:0.0309
epoch_2 [6/50], loss:0.0988, MSE_loss:0.0308
epoch_2 [7/50], loss:0.0985, MSE_loss:0.0307
epoch_2 [8/50], loss:0.0982, MSE_loss:0.0306
epoch_2 [9/50], loss:0.0980, MSE_loss:0.0305
epoch_2 [10/50], loss:0.0978, MSE_loss:0.0305
epoch_2 [11/50], loss:0.0976, MSE_loss:0.0304
epoch_2 [12/50], loss:0.0974, MSE_loss:0.0304
epoch_2 [13/50], loss:0.0972, MSE_loss:0.0303
epoch_2 [14/50], loss:0.0971, MSE_loss:0.0303
epoch_2 [15/50], loss:0.0970, MSE_loss:0.0302
epoch_2 [16/50], loss:0.0970, MSE_loss:0.0302
epoch_2 [17/50], loss:0.0968, MSE_loss:0.0302
epoch_2 [18/50], loss:0.0967, MSE_loss:0.0301
epoch_2 [19/50], loss:0.0966, MSE_loss:0.0301
epoch_2 [20/50], loss:0.0965, MSE_loss:0.0300
epoch_2 [21/50], loss:0.0965, MSE_loss:0.0300
epoch_2 [22/50], loss:0.0964, MSE_loss:0.03

In [1]:
import torch
x=torch.randn(2,3)
print(x)
print(x.view(-1))

tensor([[-1.7266,  2.2476, -1.8607],
        [-1.6655,  0.2462,  0.6537]])
tensor([-1.7266,  2.2476, -1.8607, -1.6655,  0.2462,  0.6537])


In [4]:
alist=[i+1 for i in range(50)]
alist

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]